In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,SGDRegressor,ElasticNet, LogisticRegression
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.utils.validation import check_array 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor, RandomForestClassifier, GradientBoostingClassifier

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading and pre-processing Datasets

In [ ]:
h = pd.read_csv(r'/content/drive/MyDrive/WiDS/DataSets/matches.csv')
h.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [ ]:
h['city'] = h['city'].fillna('Dubai')
h = h.replace({'Rising Pune Supergiants': 'Rising Pune Supergiant'}, regex=True)
h = h[h.winner.notna()]
h = h.drop(['umpire1','umpire2','umpire3'], axis=1)
h = h.drop(h.loc[h.season==2009].index)

In [ ]:
h = h.drop(h.loc[h.season.isin([2016,2017])].index)

In [ ]:
h = h.drop(h[h.city.isin(['Ahmedabad','Cuttack', 'Nagpur','Dharamsala','Indore', 'Visakhapatnam','Raipur','Ranchi', 'Abu Dhabi', 'Sharjah', 'Dubai'])].index)

In [ ]:
h.isna().sum()

id                 0
season             0
city               0
date               0
team1              0
team2              0
toss_winner        0
toss_decision      0
result             0
dl_applied         0
winner             0
win_by_runs        0
win_by_wickets     0
player_of_match    0
venue              0
dtype: int64

In [ ]:
h.city.unique()

array(['Bangalore', 'Chandigarh', 'Delhi', 'Mumbai', 'Kolkata', 'Jaipur',
       'Hyderabad', 'Chennai', 'Kochi', 'Pune', 'Mohali', 'Bengaluru'],
      dtype=object)

In [ ]:
h.season.unique()

array([2008, 2010, 2011, 2012, 2013, 2014, 2015, 2018, 2019])

In [ ]:
h.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue
59,60,2008,Bangalore,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Kolkata Knight Riders,140,0,BB McCullum,M Chinnaswamy Stadium
60,61,2008,Chandigarh,2008-04-19,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,bat,normal,0,Chennai Super Kings,33,0,MEK Hussey,"Punjab Cricket Association Stadium, Mohali"
61,62,2008,Delhi,2008-04-19,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,bat,normal,0,Delhi Daredevils,0,9,MF Maharoof,Feroz Shah Kotla
62,63,2008,Mumbai,2008-04-20,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,normal,0,Royal Challengers Bangalore,0,5,MV Boucher,Wankhede Stadium
63,64,2008,Kolkata,2008-04-20,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,bat,normal,0,Kolkata Knight Riders,0,5,DJ Hussey,Eden Gardens


Selecting relevant features, and one-hot encoding of categorical features

In [ ]:
dataset = h[['city','team1','team2','toss_winner','toss_decision','winner']]

dataset.head()

,city,team1,team2,toss_winner,toss_decision,winner
59,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,Kolkata Knight Riders
60,Chandigarh,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,bat,Chennai Super Kings
61,Delhi,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,bat,Delhi Daredevils
62,Mumbai,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore
63,Kolkata,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,bat,Kolkata Knight Riders


In [ ]:
dummy = pd.get_dummies(h['team1']) + pd.get_dummies(h['team2']) + pd.get_dummies(h['toss_winner'])
dataset = pd.concat([dataset, dummy], axis=1)
dataset = dataset.drop(['team1','team2','toss_winner'], axis=1)
dataset.head()

,city,toss_decision,winner,Chennai Super Kings,Deccan Chargers,Delhi Capitals,Delhi Daredevils,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Royal Challengers Bangalore,Sunrisers Hyderabad
59,Bangalore,field,Kolkata Knight Riders,0,0,0,0,0,0,1,0,0,0,2,0
60,Chandigarh,bat,Chennai Super Kings,2,0,0,0,1,0,0,0,0,0,0,0
61,Delhi,bat,Delhi Daredevils,0,0,0,1,0,0,0,0,0,2,0,0
62,Mumbai,bat,Royal Challengers Bangalore,0,0,0,0,0,0,0,2,0,0,1,0
63,Kolkata,bat,Kolkata Knight Riders,0,2,0,0,0,0,1,0,0,0,0,0


In [ ]:
dataset = pd.concat([dataset, pd.get_dummies(dataset['city'], drop_first=True)], axis=1)
dataset = dataset.drop(['city'], axis=1)
dataset.head()

,toss_decision,winner,Chennai Super Kings,Deccan Chargers,Delhi Capitals,Delhi Daredevils,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Royal Challengers Bangalore,Sunrisers Hyderabad,Bengaluru,Chandigarh,Chennai,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mohali,Mumbai,Pune
59,field,Kolkata Knight Riders,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
60,bat,Chennai Super Kings,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
61,bat,Delhi Daredevils,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0
62,bat,Royal Challengers Bangalore,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
63,bat,Kolkata Knight Riders,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
encoder = LabelEncoder()
dataset['toss_decision'] = encoder.fit_transform(dataset['toss_decision'])
dataset['winner'] = encoder.fit_transform(dataset['winner'])
dataset.head()

,toss_decision,winner,Chennai Super Kings,Deccan Chargers,Delhi Capitals,Delhi Daredevils,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Royal Challengers Bangalore,Sunrisers Hyderabad,Bengaluru,Chandigarh,Chennai,Delhi,Hyderabad,Jaipur,Kochi,Kolkata,Mohali,Mumbai,Pune
59,1,6,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
60,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
61,0,3,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0
62,0,10,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
63,0,6,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
X = pd.concat([dataset.iloc[:, :1], dataset.iloc[:,2:]], axis=1)
y = dataset.iloc[:,1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
y_train

64      9
376     7
693     6
650     6
687     3
       ..
698     2
453     0
480    10
417     0
517     6
Name: winner, Length: 400, dtype: int64

**Logistic Regression to predict match-winner**

In [ ]:
lr = LogisticRegression()

model = lr.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

0.6225

In [ ]:
model.score(X_test, y_test)

0.6039603960396039

In [ ]:
prediction = model.predict(X_test)

d = [[a,b] for a,b in zip(y_test, prediction)]
op = pd.DataFrame(d, columns=['Y_Test','Prediction'])
op

,Y_Test,Prediction
0,7,7
1,9,9
2,7,7
3,7,7
4,0,11
...,...,...
96,10,6
97,10,10
98,3,0
99,10,0


In [ ]:
acc = [x-y for x,y in zip(y_test, prediction)]
count = 0

for a in acc :
  if a==0 :
    count += 1
accuracy = count*100 / len(acc)
accuracy

60.396039603960396

**Support Vector Classifier to predict match-winner**

In [ ]:
sv = SVC()

model2 = sv.fit(X_train, y_train)

In [ ]:
model2.score(X_train, y_train)

0.6825

In [ ]:
model2.score(X_test, y_test)

0.5148514851485149

In [ ]:
prediction = model2.predict(X_test)

d = [[a,b] for a,b in zip(y_test, prediction)]
op = pd.DataFrame(d, columns=['Y_Test','Prediction'])
op

,Y_Test,Prediction
0,7,7
1,9,9
2,7,7
3,7,7
4,0,11
...,...,...
96,10,6
97,10,10
98,3,0
99,10,0


In [ ]:
acc = [x-y for x,y in zip(y_test, prediction)]
count = 0

for a in acc :
  if a==0 :
    count += 1
accuracy = count*100 / len(acc)
accuracy

51.48514851485149

**Random Forest Classifier to predict match-winner**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid_rf2 = {
              "n_estimators": [100,300,500],
              "max_depth": [10,50,100],           
             }         
gr_rf2 = GridSearchCV(RandomForestClassifier(),param_grid_rf2,cv=5,verbose=2)
gr_rf2.fit(X_train,y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END .....................max_depth=10, n_estimators=100; total time=   0.2s
[CV] END .....................max_depth=10, n_estimators=100; total time=   0.3s
[CV] END .....................max_depth=10, n_estimators=100; total time=   0.4s
[CV] END .....................max_depth=10, n_estimators=100; total time=   0.3s
[CV] END .....................max_depth=10, n_estimators=100; total time=   0.3s
[CV] END .....................max_depth=10, n_estimators=300; total time=   0.9s
[CV] END .....................max_depth=10, n_estimators=300; total time=   0.9s
[CV] END .....................max_depth=10, n_estimators=300; total time=   0.5s
[CV] END .....................max_depth=10, n_estimators=300; total time=   0.5s
[CV] END .....................max_depth=10, n_estimators=300; total time=   0.5s
[CV] END .....................max_depth=10, n_estimators=500; total time=   0.8s
[CV] END .....................max_depth=10, n_est

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 50, 100],
                         'n_estimators': [100, 300, 500]},
             verbose=2)

In [ ]:
gr_rf2.best_params_

{'max_depth': 10, 'n_estimators': 100}

In [ ]:
rf_best2 = RandomForestClassifier(random_state=300,max_features=None,
                                    n_estimators=gr_rf2.best_params_['n_estimators'],
                                    max_depth = gr_rf2.best_params_['max_depth'], verbose=2
                                    )
rf_best2.fit(X_train,y_train)
y_pred = rf_best2.predict(X_test)
score = rf_best2.score(X_test,y_test)

acc = [x-y for x,y in zip(y_test, y_pred)]
count = 0

for a in acc :
  if a==0 :
    count += 1
accuracy = count*100 / len(acc)

print("Score :", score)
print('Accuracy :', accuracy)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [ ]:
y_pred

array([ 4,  9,  7,  0,  0,  7,  6, 10, 11,  6,  7,  9,  0,  6,  9,  7,  9,
        7,  3,  7,  9,  6,  7,  6,  7,  0,  0,  9, 10,  6,  4,  6, 10,  9,
        0,  9,  9,  0,  0,  9,  3,  0,  7,  0,  4,  6,  3,  7,  4,  2,  3,
       10,  6,  0,  7,  6,  3, 10,  7, 11,  6,  0,  4, 10,  0,  0,  0, 11,
       11,  1,  0, 11, 10,  0,  0,  6,  0,  4,  6,  0,  6,  6, 11, 10,  4,
        7, 11, 11,  0,  0,  6,  0,  0,  9, 10,  6,  6, 10,  0,  0,  9])

**Gradient Boosting Classifier to predict match-winner**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid_rf = {
              #"n_estimators": [200,300],
              "learning_rate": [0.25,0.5,0.75],
              "max_depth": [10,14,18],           
             }         
gr_rf = GridSearchCV(GradientBoostingClassifier(),param_grid_rf,cv=5,verbose=2)
gr_rf.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ...................learning_rate=0.25, max_depth=10; total time=   4.5s
[CV] END ...................learning_rate=0.25, max_depth=10; total time=   2.9s
[CV] END ...................learning_rate=0.25, max_depth=10; total time=   3.1s
[CV] END ...................learning_rate=0.25, max_depth=10; total time=   3.0s
[CV] END ...................learning_rate=0.25, max_depth=10; total time=   3.0s
[CV] END ...................learning_rate=0.25, max_depth=14; total time=   3.8s
[CV] END ...................learning_rate=0.25, max_depth=14; total time=   3.5s
[CV] END ...................learning_rate=0.25, max_depth=14; total time=   3.6s
[CV] END ...................learning_rate=0.25, max_depth=14; total time=   3.7s
[CV] END ...................learning_rate=0.25, max_depth=14; total time=   3.5s
[CV] END ...................learning_rate=0.25, max_depth=18; total time=   3.4s
[CV] END ...................learning_rate=0.25, m

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.25, 0.5, 0.75],
                         'max_depth': [10, 14, 18]},
             verbose=2)

In [ ]:
gr_rf.best_params_

{'learning_rate': 0.25, 'max_depth': 10}

In [ ]:
rf_best = GradientBoostingClassifier(random_state=300,
                                    n_estimators =5000,
                                    learning_rate=gr_rf.best_params_['learning_rate'],
                                    max_depth = gr_rf.best_params_['max_depth'], verbose=2
                                    )
rf_best.fit(X_train,y_train)
y_pred = rf_best.predict(X_test)
score = rf_best.score(X_test,y_test)

acc = [x-y for x,y in zip(y_test, y_pred)]
count = 0

for a in acc :
  if a==0 :
    count += 1
accuracy = count*100 / len(acc)

print("Score :", score)
print('Accuracy :', accuracy)

Streaming output truncated to the last 5000 lines.
         3           0.5070            4.80m
         4           0.4147            5.23m
         5           0.3575            5.54m
         6           0.3191            5.72m
         7           0.2909            5.94m
         8           0.2711            6.08m
         9           0.2571            6.25m
        10           0.2478            6.37m
        11           0.2403            6.49m
        12           0.2354            6.58m
        13           0.2315            6.83m
        14           0.2287            6.87m
        15           0.2264            6.90m
        16           0.2247            6.98m
        17           0.2236            7.03m
        18           0.2226            7.04m
        19           0.2219            7.13m
        20           0.2213            7.14m
        21           0.2209            7.16m
        22           0.2206            7.20m
        23           0.2204            7.21m
    